In [8]:
!pip install psycopg2-binary
!pip install shortuuid

You should consider upgrading via the '/Users/KimJungHwan/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/KimJungHwan/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [9]:
import requests
import json
from requests.auth import HTTPBasicAuth
import datetime
import random
import pandas as pd
import uuid 
import psycopg2
import DB
import shortuuid


## 가입 때 필요한 정보

In [10]:
AccessToken = "79f94971b3be510a39f475a10e5e0ccd8a096c4316c04c12b9cbce0c0dfc7f7a"
#기업코드
Iscd = "000704"
#오늘 날짜
Tsymd = datetime.datetime.now().strftime("%Y%m%d")
#생년월일
BrdtBrno = "19501211"
Bncd = "011"
Acno = "3020000003001"
FinAcno = "00820100007040000000000005724"
random_sequence = "".join([str(i) for i in list(random.sample([0,1,2,3,4,5,6,7,8,9], 9))])

In [11]:
itsme_info = {"AccessToken" : AccessToken, 'Iscd' : Iscd, "BrdtBrno" : BrdtBrno, "Bncd" : Bncd, "Acno" : Acno, "FinAcno" : FinAcno }


In [12]:
AccessToken = "51904849dc027093a8ed3072e4a7389b3b30da24fe3225be2527d8f14258d05b"
#기업코드
Iscd = "000751"
#오늘 날짜
Tsymd = datetime.datetime.now().strftime("%Y%m%d")
#생년월일
BrdtBrno = "19501213"
Bncd = "011"
Acno = "3020000003229"
FinAcno = "00820100007510000000000005759"
random_sequence = "".join([str(i) for i in list(random.sample([0,1,2,3,4,5,6,7,8,9], 9))])

In [13]:
user_info = {"AccessToken" : AccessToken, 'Iscd' : Iscd, "BrdtBrno" : BrdtBrno, "Bncd" : Bncd, "Acno" : Acno, "FinAcno" : FinAcno }

## 출금
### 이츠미 계정에서 출석체크가 있을 때마다 출금 

In [14]:
def withdrawl(user, amount, message = ""):

    url_items = "https://developers.nonghyup.com/DrawingTransfer.nh"
    headers = {'Content-type':'application/json', 'Accept':'application/json'}
    
    
    AccessToken = user['AccessToken']
    #기업코드
    Iscd = user['Iscd']
    #생년월일
    BrdtBrno = user['BrdtBrno']
    Bncd = user['Bncd']
    Acno = user["Acno"]    
    FinAcno = user["FinAcno"] 
    
    Tsymd = datetime.datetime.now().strftime("%Y%m%d")
    Trtm = datetime.datetime.now().strftime("%H%M%S")
    IsTuno = "".join([str(i) for i in list(random.sample([0,1,2,3,4,5,6,7,8,9], 9))])
        
    
    newItem = {"Header": {
        "ApiNm": "DrawingTransfer",
        "Tsymd": Tsymd,
        "Trtm": Trtm,
        "Iscd": Iscd,
        "FintechApsno": "001",
        "ApiSvcCd": "DrawingTransferA",
        "IsTuno": IsTuno,
        "AccessToken": AccessToken
          },
  "FinAcno": FinAcno,
  "Tram": amount,
  "DractOtlt": "test",
  "MractOtlt":"테스트"

}
    
    r = requests.post(url_items, auth=HTTPBasicAuth('shany.ka', 'shanky1213'), json=newItem, headers=headers)

    r_text= json.loads(r.text)
    
    if '정상' in r_text['Header']['Rsms'] :
        return {'uuid': shortuuid.uuid(), 'trade_time' : int(Tsymd+Trtm)}
    else :
        raise ValueError(str(r_text['Header']['Rsms']))
    


In [15]:
withdrawl(itsme_info, 1)

{'uuid': 'GGCEcgm4zCyRTL4GQZUCRF', 'trade_time': 20201213162102}

## Deposit
### 가입 계정의 계좌번호에 입금 (모계좌 -> 자계좌)

In [47]:
def deposit(user, amount, message = "university_attendance"):
    
    url_items = "https://developers.nonghyup.com/ReceivedTransferAccountNumber.nh"
    headers = {'Content-type':'application/json', 'Accept':'application/json'}
    
    AccessToken = user['AccessToken']
    #기업코드
    Iscd = user['Iscd']
    #생년월일
    BrdtBrno = user['BrdtBrno']
    Bncd = user['Bncd']
    Acno = user["Acno"]

    Tsymd = datetime.datetime.now().strftime("%Y%m%d")
    IsTuno = "".join([str(i) for i in list(random.sample([0,1,2,3,4,5,6,7,8,9], 9))])
    Trtm = datetime.datetime.now().strftime("%H%M%S")
    
    newItem = {
      "Header": {
        "ApiNm": "ReceivedTransferAccountNumber",
        "Tsymd":Tsymd,
        "Trtm":Trtm,
        "Iscd": Iscd,
        "FintechApsno":"001",
        "ApiSvcCd":"ReceivedTransferA",
        "IsTuno": IsTuno,
        "AccessToken": AccessToken
            },
      "Bncd": "011",
      "Acno": Acno,
      "Tram": amount,
      "DractOtlt": message,
      "MractOtlt": message
    }    
    
    r = requests.post(url_items, auth=HTTPBasicAuth('shany.ka', 'shanky1213'), json=newItem, headers=headers)

    r_text= json.loads(r.text)
    print(r.text)
    print(r.status_code)
    print(r_text)

In [38]:
deposit(user_info, 1)

{"Header":{"Trtm":"170045","Rsms":"정상처리 되었습니다.","ApiNm":"ReceivedTransferAccountNumber","IsTuno":"073418269","Tsymd":"20201213","FintechApsno":"001","Iscd":"000751","Rpcd":"00000","ApiSvcCd":"ReceivedTransferA"}}
200
{'Header': {'Trtm': '170045', 'Rsms': '정상처리 되었습니다.', 'ApiNm': 'ReceivedTransferAccountNumber', 'IsTuno': '073418269', 'Tsymd': '20201213', 'FintechApsno': '001', 'Iscd': '000751', 'Rpcd': '00000', 'ApiSvcCd': 'ReceivedTransferA'}}


# 거래내역 확인
## 유저의 자계좌의 거래내역 확인

In [39]:
def check_trades(user, start_time =''):
    headers = {'Content-type':'application/json', 'Accept':'application/json'}
    url_items = "https://developers.nonghyup.com/InquireTransactionHistory.nh"

    
    AccessToken = user['AccessToken']
    #기업코드
    Iscd = user['Iscd']
    #생년월일
    BrdtBrno = user['BrdtBrno']
    Bncd = user['Bncd']
    Acno = user["Acno"]    
    
    Tsymd = datetime.datetime.now().strftime("%Y%m%d")
    IsTuno = "".join([str(i) for i in list(random.sample([0,1,2,3,4,5,6,7,8,9], 9))])
    Trtm = datetime.datetime.now().strftime("%H%M%S")
    
    newItem = {
    "Header":{
        "ApiNm":"InquireTransactionHistory",
        "Tsymd":Tsymd,
        "Trtm":Trtm,
        "Iscd": Iscd,
        "FintechApsno":"001",
        "ApiSvcCd":"ReceivedTransferA",
        "IsTuno":IsTuno,
        "AccessToken":
        AccessToken},
        "Bncd":Bncd,
        "Acno":Acno,
        "Insymd":"20201201",
        "Ineymd":Tsymd,
        "TrnsDsnc":"A",
        "Lnsq":"DESC",
        "PageNo":"1",
        "Dmcnt":"100"
    }


    r = requests.post(url_items, auth=HTTPBasicAuth('shany.ka', 'shanky1213'), json=newItem, headers=headers)
    r_text= json.loads(r.text)
#     print(r.status_code)

#     print(r_text)    
    a = pd.DataFrame(r_text['REC'])
    a['trade_time'] = a['Trdd'] + a['Txtm']
    a['trade_time'] = a['trade_time'].map(int)
    a.sort_values(by=['Trdd', 'Txtm'], axis=0, inplace=True, ascending = False)
    if start_time != '':
        a = a[a['trade_time']>= start_time]
    return a



In [40]:
check_trades(user_info, 20201213160000)

,AftrBlnc,TrnsAfAcntBlncSmblCd,BnprCntn,Txtm,Tuno,Trdd,Smr,Ccyn,MnrcDrotDsnc,Tram,HnisCd,HnbrCd,trade_time
23,27,+,joklip,170045,21973,20201213,None,0,2,1,000,0000000,20201213170045
17,26,+,haha,170034,21971,20201213,None,0,2,1,000,0000000,20201213170034
11,25,+,haha,170004,21968,20201213,None,0,2,1,000,0000000,20201213170004
4,24,+,42ACR7MDVmJGqsdUmVGaDc,162215,21618,20201213,None,0,2,1,000,0000000,20201213162215
22,23,+,haha,162110,21614,20201213,None,0,2,1,000,0000000,20201213162110
16,22,+,TNY6ercpTcmiA8W67NcYRZ,160617,21484,20201213,None,0,2,1,000,0000000,20201213160617
10,21,+,3ZuUDLxdMPMpG8e6Tycibh,160553,21476,20201213,None,0,2,1,000,0000000,20201213160553
3,20,+,haha,160550,21472,20201213,None,0,2,1,000,0000000,20201213160550
21,19,+,6NAnp8qG2fdkXYZZJNAbkN,160435,21453,20201213,None,0,2,1,000,0000000,20201213160435
15,18,+,aZLkzkcjLAbYJDjpYCDFx2,160418,21445,20201213,None,0,2,1,000,0000000,20201213160418


## 계정의 잔고 확인
### 자계좌의 잔고 확인

In [41]:
def check_balance(user):
    headers = {'Content-type':'application/json', 'Accept':'application/json'}
    url_items = "https://developers.nonghyup.com/InquireBalance.nh"

    
    AccessToken = user['AccessToken']
    #기업코드
    Iscd = user['Iscd']
    #생년월일
    BrdtBrno = user['BrdtBrno']
    Bncd = user['Bncd']
    Acno = user["Acno"]    
    FinAcno = user["FinAcno"] 
    
    Tsymd = datetime.datetime.now().strftime("%Y%m%d")
    IsTuno = "".join([str(i) for i in list(random.sample([0,1,2,3,4,5,6,7,8,9], 9))])
    
    newItem = {
        "Header": {
            "ApiNm": "InquireBalance",
            "Tsymd": Tsymd,
            "Trtm": "112428",
            "Iscd": Iscd,
            "FintechApsno": "001",
            "ApiSvcCd": "ReceivedTransferA",
            "IsTuno": IsTuno,
            "AccessToken": AccessToken
        },
        "FinAcno": FinAcno
    }

    r = requests.post(url_items, auth=HTTPBasicAuth('shany.ka', 'shanky1213'), json=newItem, headers=headers)
    r_text= json.loads(r.text)
    print(r.status_code)
    print(r_text)        
    return r_text['Ldbl']

In [42]:
check_balance(user_info)


200
{'FinAcno': '00820100007510000000000005759', 'Header': {'Trtm': '112428', 'Rsms': '정상처리 되었습니다.', 'ApiNm': 'InquireBalance', 'IsTuno': '194875036', 'Tsymd': '20201213', 'FintechApsno': '001', 'Iscd': '000751', 'Rpcd': '00000', 'ApiSvcCd': 'ReceivedTransferA'}, 'Ldbl': '27', 'RlpmAbamt': '27'}


'27'

# 출석 체크
### 출석체크와 동시에 trade_id DB에 입력 및 user_balance 반환

In [23]:
def check(itsme_info, user_info):
    a = DB.DBBot("NH")
    trade = withdrawl(itsme_info, 1)
    trade_id = trade['uuid']
    trade_time = trade['trade_time']
    deposit(user_info, 1, trade_id)
    user_balance = check_balance(user_info)
    a.insert_order(trade_id,user_info['Iscd'], trade_time)
    print(trade_time)
    return user_balance

In [52]:
check(itsme_info, user_info)

{"Header":{"Trtm":"170758","Rsms":"정상처리 되었습니다.","ApiNm":"ReceivedTransferAccountNumber","IsTuno":"128506437","Tsymd":"20201213","FintechApsno":"001","Iscd":"000751","Rpcd":"00000","ApiSvcCd":"ReceivedTransferA"}}
200
{'Header': {'Trtm': '170758', 'Rsms': '정상처리 되었습니다.', 'ApiNm': 'ReceivedTransferAccountNumber', 'IsTuno': '128506437', 'Tsymd': '20201213', 'FintechApsno': '001', 'Iscd': '000751', 'Rpcd': '00000', 'ApiSvcCd': 'ReceivedTransferA'}}
200
{'FinAcno': '00820100007510000000000005759', 'Header': {'Trtm': '112428', 'Rsms': '정상처리 되었습니다.', 'ApiNm': 'InquireBalance', 'IsTuno': '834065719', 'Tsymd': '20201213', 'FintechApsno': '001', 'Iscd': '000751', 'Rpcd': '00000', 'ApiSvcCd': 'ReceivedTransferA'}, 'Ldbl': '33', 'RlpmAbamt': '33'}
20201213170758


'33'

# 데이터베이스



In [25]:
a = DB.DBBot("NH")

In [26]:
a.get_connection()

(<connection object at 0x7fd6e883d730; dsn: 'user=NH password=xxx dbname=NH host=211.218.53.149 port=5432', closed: 0>,
 <cursor object at 0x7fd6e61c2650; closed: 0>)

# 특정 시간 이후의 내용 가져올 수 있음

In [53]:
# Iscd = 000751
# start id
a.get_trade_ids("000751", 20201211160000)

,trade_id,iscd,trade_time
0,6NAnp8qG2fdkXYZZJNAbkN,000751,20201213160435
1,3ZuUDLxdMPMpG8e6Tycibh,000751,20201213160553
2,TNY6ercpTcmiA8W67NcYRZ,000751,20201213160617
3,42ACR7MDVmJGqsdUmVGaDc,000751,20201213162215
4,fVArA5SnuP4enetThe4F73,000751,20201213170234
5,AGusbXdxshwKwNmmAC6A8i,000751,20201213170753
6,T9auexLzviATLZkUqZyjem,000751,20201213170754
7,aUfTixjve5FAfA6uEpMgve,000751,20201213170755
8,RtN57ipdCuYQ6w2zMqHtST,000751,20201213170757
9,gza57qxStuRj7D7GxGbj64,000751,20201213170758


*******Error********unrecognized configuration parameter "tables"



In [30]:
class DBBot():
    def __init__(self, name)
    def get_connect_info(self)
    def get_connection(self):
    def execute_query(self, query):
    def insert_order(self, trade_id, Iscd):
    def insert_order(self, trade_id, Iscd, trade_time):
    def get_trade_ids(self, Iscd, start_time = ''):

SyntaxError: invalid syntax (<ipython-input-30-2c77b158f5d7>, line 2)

In [ ]:
def withdrawl(user, amount, message = ""):
def deposit(user, amount, message = "haha"):


In [ ]:
# 이츠미 계좌에서 유저한테
# 리턴 값은 유저 잔액
def check(itsme_info, user_info):
